In [ ]:
import sys
import time
import numpy as np
import pandas as pd
import glob as glob
import os
import itertools

from scipy.interpolate import interp1d

from astropy.io import fits
from astropy.visualization import simple_norm
from astropy.wcs import WCS
from astropy.modeling import models, fitting
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.table import Table, QTable, vstack
from astropy import units as u
from astropy.coordinates import SkyCoord

from astropy.nddata.utils import Cutout2D

from scipy.interpolate import interp1d
from scipy.stats import gaussian_kde
from scipy.ndimage import gaussian_filter

In [ ]:
%matplotlib inline
from matplotlib import style, pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as ticker
import matplotlib.colors as col
from matplotlib.colors import ListedColormap
import seaborn as sb
import matplotlib.gridspec as gridspec
sb.set_style('white')
from matplotlib.ticker import (MultipleLocator, AutoLocator, AutoMinorLocator)

from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams['image.cmap'] = 'jet'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['figure.figsize'] = (7,5)
plt.rcParams['axes.titlesize'] = plt.rcParams['axes.labelsize'] = 25
plt.rcParams['xtick.labelsize'] = plt.rcParams['ytick.labelsize'] = 25

font1 = {'family': 'sans-serif', 'color': 'black', 'weight': 'normal', 'size': '15'}
font2 = {'family': 'sans-serif', 'color': 'black', 'weight': 'normal', 'size': '25'}

In [ ]:
import pista as pt
data_path = pt.data_dir

# **Input Data**

## **JWST**

In [ ]:
dict_images_jwst = {}

dict_cats = {}

images_dir = '../data/NGC_628/JWST/'

images = sorted(glob.glob(os.path.join(images_dir, "*/*i2d.fits")))
cats  = sorted(glob.glob(os.path.join(images_dir, "*/*cat.ecsv")))

for image in images:

    im = fits.open(image)
    f = im[0].header['FILTER']
    d = im[0].header['DETECTOR']

    if d == 'MULTIPLE':
        d = 'NRCB3'

    # Image
    if d not in dict_images_jwst.keys():
        dict_images_jwst[d] = {}
        
    if f not in dict_images_jwst[d].keys():
        dict_images_jwst[d][f] =  {'images': [image]}
    else:
        dict_images_jwst[d][f]['images'].append(image)
    
    # Catalog
    if d not in dict_cats.keys():
        dict_cats[d] = {}
        
    if f not in dict_cats[d].keys():
        dict_cats[d][f] =  {'cats': [image[:-8] + 'cat.ecsv']}
    else:
        dict_cats[d][f]['cats'].append(image[:-8] + 'cat.ecsv')
        
 
print('Available Detectors and Filters\n-------------------------------')
      
for i in dict_images_jwst.keys():
    print(f'{i} :', list(dict_images_jwst[i].keys()))

## **HST**

In [ ]:
images_dir = '../data/NGC_628/HST/'
images = sorted(glob.glob(os.path.join(images_dir, "*/*drc.fits")))

dict_images_hst = {}

for image in images:

    im = fits.open(image)
    if 'FILTER2' in im[0].header.keys():
        f = im[0].header['FILTER2']
    elif 'FILTER' in im[0].header.keys():
        f = im[0].header['FILTER']
    
    if 'CLEAR' in f:
        f = im[0].header['FILTER1']
        
    d = im[0].header['DETECTOR']

    # Image
    if d not in dict_images_hst.keys():
        dict_images_hst[d] = {}
        
    if f not in dict_images_hst[d].keys():
        dict_images_hst[d][f] =  {'images': [image]}
    else:
        dict_images_hst[d][f]['images'].append(image)    
 
print('Available Detectors and Filters\n-------------------------------')
      
for i in dict_images_hst.keys():
    print(f'{i} :', list(dict_images_hst[i].keys()))

# **Functions**

In [ ]:
dict_images = {}
dict_images.update(dict_images_jwst)
dict_images.update(dict_images_hst)

In [ ]:
def gen_cutout(ra=24.1859050, dec=15.7726111,size=50, det_n='NRCB3',filt_n='F115W',
              fig=None, figsize=(30,20)):
    # MIRI
    det_m = 'MIRIMAGE'
    filt_m = 'F770W'

    hdul_m = fits.open(dict_images[det_m][filt_m]['images'][0])
    data_m = hdul_m[1].data

    imh_m = hdul_m[1].header
    wcs_m = WCS(imh_m)
    hdul_m.close()

    hdul_n = fits.open(dict_images[det_n][filt_n]['images'][0])
    data_n = hdul_n[1].data

    imh_n = hdul_n[1].header
    wcs_n = WCS(imh_n)

    hdul_n.close()
    
    pos = SkyCoord(ra=ra, dec=dec, unit='deg')
    cutout_m = Cutout2D(data_m,pos,size = 50*u.arcsec, wcs = wcs_m)
    cutout_n = Cutout2D(data_n,pos,size = 50*u.arcsec, wcs = wcs_n)
    
    if fig is None:
        fig = plt.figure(figsize=figsize)

    ax1 = fig.add_subplot(1, 2, 1, projection=cutout_m.wcs)

    ax1.set_title(filt_m, fontdict=font2)
    norm = simple_norm(cutout_m.data, 'sqrt',min_cut=13, max_cut = 40)

    img = ax1.imshow(cutout_m.data, norm = norm, cmap='gray')
    # MIRI Contours
    levels = [13.2,15]
    cmap = ListedColormap(['yellow','cyan'])
    smooth_data = gaussian_filter(cutout_m.data, 2)
    ax1.contour(smooth_data, levels=levels, cmap=cmap)

    cb = plt.colorbar(img, orientation='horizontal', anchor = (0.5, 1.5))
    cb.set_label(f"{imh_m['BUNIT']}")

    ax2 = fig.add_subplot(1, 2, 2, projection=cutout_n.wcs)

    ax2.set_title(filt_n, fontdict=font2)
    norm = simple_norm(cutout_n.data, 'sqrt',percent=99.)

    img = ax2.imshow(cutout_n.data, norm = norm, cmap='gray')
    ax2.contour(smooth_data, levels=levels, cmap=cmap, transform=ax2.get_transform(cutout_m.wcs))
    cb = plt.colorbar(img, orientation='horizontal', anchor = (0.5, 1.5))
    cb.set_label(f"{imh_n['BUNIT']}")
    
    return fig, [ax1, ax2]

In [ ]:
def NIRCAM_phot(out_dir,region='bubble', filt_n='F115W', d=50, comp=True, 
               skip_phot=False):

    ra, dec = regions[region]['ra'], regions[region]['dec']

    det_n = 'NRCB3'
    
    input_path = dict_images[det_n][filt_n]['images'][0]
    
    if not skip_phot:
        if not os.path.exists(f"../{out_dir}/{region}/"):
            os.mkdir(f"../{out_dir}/{region}")

        if not os.path.exists(f"../{out_dir}/{region}/{filt_n}/"):
            os.mkdir(f"../{out_dir}/{region}/{filt_n}")

        # Generating copy of data
        os.system(f"cp {input_path} ../{out_dir}/{region}/{filt_n}/data.fits")

        # Applying NIRCAMMASK
        os.system(f"nircammask ../{out_dir}/{region}/{filt_n}/data.fits")

        # Generating cutout
        os.system(f"python ../scripts/dolphot_convert_nircam.py --f ../{out_dir}/{region}/{filt_n}/data.fits --d 'NRCB3' --c True --ra {ra} --dec {dec} --radius {d}")

        # Removing copy
        os.remove(f"../{out_dir}/{region}/{filt_n}/data.fits")
        
        # Calculating Sky
        os.system(f"calcsky ../{out_dir}/{region}/{filt_n}/data_conv 10 25 2 2.25 2.00")

        # Editing DOLPHOT parameter file
        with open("../params/nircam_dolphot.param") as f:
            dat = f.readlines()

        dat[4] = f'img1_file = ../{out_dir}/{region}/{filt_n}/data_conv            #image 1\n'

        with open("../params/nircam_dolphot.param", 'w', encoding='utf-8') as f:
            f.writelines(dat)

        # Running DOLPHOT
        os.system(f"dolphot ../{out_dir}/{region}/{filt_n}/out -p../params/nircam_dolphot.param")

        # Filtering Output photometric catalog

        filter_phot(out_dir, region, filt_n)

    if comp:
        n = 100
        counts = []
        mags = np.arange(20,30, 0.5)
        for mag in mags:
            # Completeness Analysis
            sim_stars(out_dir, region, filt_n, d, mag,n)
            # Calculating Sky
            os.system(f"calcsky ../PHOT_OUTPUT_m50/{region}/{filt_n}/data_conv 10 25 2 2.25 2.00")

            # Editing DOLPHOT parameter file
            with open("../params/nircam_dolphot.param") as f:
                dat = f.readlines()

            dat[4] = f'img1_file = ../PHOT_OUTPUT_m50/{region}/{filt_n}/data_conv            #image 1\n'

            with open("../params/nircam_dolphot.param", 'w', encoding='utf-8') as f:
                f.writelines(dat)

            # Running DOLPHOT
            os.system(f"dolphot ../PHOT_OUTPUT_m50/{region}/{filt_n}/out -p../params/nircam_dolphot.param")

            # Filtering Output photometric catalog

            filter_phot('PHOT_OUTPUT_m50', region, filt_n)

            counts.append(comp_base(out_dir, region, filt_n, 0.06, mag))
        
        x = mags
        y = np.array(counts)/n
        df = pd.DataFrame(zip(x,y), columns=['mag','frac'])
        df.to_csv(f"../{out_dir}/{region}/{filt_n}/comp_{filt_n}.csv")
        comp_fit(out_dir, region, filt_n, x,y)

In [ ]:
def acs_phot(out_dir, region='bubble', filt_n='F435W', d=50, comp=True,
             skip_phot=False):

    ra, dec = regions[region]['ra'], regions[region]['dec']

    det_n = 'WFC'
    input_path = dict_images[det_n][filt_n]['images'][0]
    
    if not skip_phot:
        if not os.path.exists(f"../{out_dir}/{region}/"):
            os.mkdir(f"../{out_dir}/{region}")

        if not os.path.exists(f"../{out_dir}/{region}/{filt_n}/"):
            os.mkdir(f"../{out_dir}/{region}/{filt_n}")

        # Generating copy of data
        os.system(f"cp {input_path} ../{out_dir}/{region}/{filt_n}/data.fits")

        # Applying ACSMASK
        os.system(f"acsmask ../{out_dir}/{region}/{filt_n}/data.fits")

        # Generating cutout
        os.system(f"python ../scripts/dolphot_convert_acs.py --f ../{out_dir}/{region}/{filt_n}/data.fits --c True --ra {ra} --dec {dec} --radius {d}")
        
        # Removing copy
        os.remove(f"../{out_dir}/{region}/{filt_n}/data.fits")
        
        # Calculating Sky
        os.system(f"calcsky ../{out_dir}/{region}/{filt_n}/data_conv 15 35 4 2.25 2.00")

        # Editing DOLPHOT parameter file
        with open("../params/acs_dolphot.param") as f:
            dat = f.readlines()

        dat[4] = f'img1_file = ../{out_dir}/{region}/{filt_n}/data_conv            #image 1\n'

        with open("../params/acs_dolphot.param", 'w', encoding='utf-8') as f:
            f.writelines(dat)

        # Running DOLPHOT
        os.system(f"dolphot ../{out_dir}/{region}/{filt_n}/out -p../params/acs_dolphot.param")

        # Filtering Output photometric catalog

        filter_phot(out_dir, region, filt_n)  
    
    if comp:
        n=100
        counts = []
        mags = np.arange(22,32, 0.5)
        for mag in mags:
            # Completeness Analysis
            sim_stars(out_dir, region, filt_n, d, mag,n)

            # Calculating Sky
            os.system(f"calcsky ../PHOT_OUTPUT_m50/{region}/{filt_n}/data_conv 15 35 4 2.25 2.00")

            # Editing DOLPHOT parameter file
            with open("../params/acs_dolphot.param") as f:
                dat = f.readlines()

            dat[4] = f'img1_file = ../PHOT_OUTPUT_m50/{region}/{filt_n}/data_conv            #image 1\n'

            with open("../params/acs_dolphot.param", 'w', encoding='utf-8') as f:
                f.writelines(dat)

            # Running DOLPHOT
            os.system(f"dolphot ../PHOT_OUTPUT_m50/{region}/{filt_n}/out -p../params/acs_dolphot.param")
                # Filtering Output photometric catalog

            filter_phot('PHOT_OUTPUT_m50', region, filt_n)
            
            counts.append(comp_base(out_dir, region, filt_n, 0.10, mag))

        x = mags
        y = np.array(counts)/n
        df = pd.DataFrame(zip(x,y), columns=['mag','frac'])
        df.to_csv(f"../{out_dir}/{region}/{filt_n}/comp_{filt_n}.csv")
        comp_fit(out_dir, region, filt_n, x,y)

In [ ]:
def wfc3_phot(out_dir, region='bubble', filt_n='F275W', d=50, comp=True,
             skip_phot=False):

    ra, dec = regions[region]['ra'], regions[region]['dec']
    det_n = 'UVIS'
    input_path = dict_images[det_n][filt_n]['images'][0]
    
    if not skip_phot:
        if not os.path.exists(f"../{out_dir}/{region}/"):
            os.mkdir(f"../{out_dir}/{region}")

        if not os.path.exists(f"../{out_dir}/{region}/{filt_n}/"):
            os.mkdir(f"../{out_dir}/{region}/{filt_n}")

        # Generating copy of data
        os.system(f"cp {input_path} ../{out_dir}/{region}/{filt_n}/data.fits")

        # Applying WFC3MASK
        os.system(f"wfc3mask ../{out_dir}/{region}/{filt_n}/data.fits")

        # Generating cutout
        os.system(f"python ../scripts/dolphot_convert_wfc3.py --f ../{out_dir}/{region}/{filt_n}/data.fits --c True --ra {ra} --dec {dec} --radius {d}")
        
        # Removing copy
        os.remove(f"../{out_dir}/{region}/{filt_n}/data.fits")
        
        # Calculating Sky
        os.system(f"calcsky ../{out_dir}/{region}/{filt_n}/data_conv 15 35 4 2.25 2.00")

        # Editing DOLPHOT parameter file
        with open("../params/wfc3_dolphot.param") as f:
            dat = f.readlines()

        dat[4] = f'img1_file = ../{out_dir}/{region}/{filt_n}/data_conv            #image 1\n'

        with open("../params/wfc3_dolphot.param", 'w', encoding='utf-8') as f:
            f.writelines(dat)

        # Running DOLPHOT
        os.system(f"dolphot ../{out_dir}/{region}/{filt_n}/out -p../params/wfc3_dolphot.param")

        # Filtering Output photometric catalog

        filter_phot(out_dir, region, filt_n)
    
    if comp:
        n = 100
        counts = []
        mags = np.arange(22, 32, 0.5)
        for mag in mags:
            # Completeness Analysis
            sim_stars(out_dir, region, filt_n, d, mag,n)
        
            # Calculating Sky
            os.system(f"calcsky ../PHOT_OUTPUT_m50/{region}/{filt_n}/data_conv 15 35 4 2.25 2.00")

            # Editing DOLPHOT parameter file
            with open("../params/wfc3_dolphot.param") as f:
                dat = f.readlines()

            dat[4] = f'img1_file = ../PHOT_OUTPUT_m50/{region}/{filt_n}/data_conv            #image 1\n'

            with open("../params/wfc3_dolphot.param", 'w', encoding='utf-8') as f:
                f.writelines(dat)

            # Running DOLPHOT
            os.system(f"dolphot ../PHOT_OUTPUT_m50/{region}/{filt_n}/out -p../params/wfc3_dolphot.param")
            # Filtering Output photometric catalog

            filter_phot('PHOT_OUTPUT_m50', region, filt_n)
            counts.append(comp_base(out_dir, region, filt_n, 0.12, mag))
            
        x = mags
        y = np.array(counts)/n
        
        df = pd.DataFrame(zip(x,y), columns=['mag','frac'])
        df.to_csv(f"../{out_dir}/{region}/{filt_n}/comp_{filt_n}.csv")
        
        comp_fit(out_dir, region, filt_n, x,y)

In [ ]:
def filter_phot(out_dir, region, filt_n):
    os.system(f"python ../scripts/to_table.py --f ../{out_dir}/{region}/{filt_n}/out")
    
    phot_table = Table.read(f"../{out_dir}/{region}/{filt_n}/photometry.fits")
    phot_table.rename_columns(['mag_vega'],[f'mag_vega_{filt_n}'])
    
    hdu = fits.open(f"../{out_dir}/{region}/{filt_n}/data_conv.fits")[1]
    
    wcs = WCS(hdu.header)
    positions = np.transpose([phot_table['x'] - 0.5, phot_table['y']-0.5])
    
    coords = np.array(wcs.pixel_to_world_values(positions))
    phot_table['ra'] = coords[:,0]
    phot_table['dec'] = coords[:,1]
    
    # NIRCAM Filter Short (Warfield et.al)
    if filt_n in ['F115W', 'F150W','F200W']:
        phot_table1 = phot_table[ (phot_table['sharpness']**2 <= 0.01) &
                                    (phot_table['obj_crowd'] <=  0.5) &
                                    (phot_table['flags']     <=    2) &
                                    (phot_table['type']      <=    2)]

        phot_table2 = phot_table[ ~((phot_table['sharpness']**2 <= 0.01) &
                                    (phot_table['obj_crowd'] <=  0.5) &
                                    (phot_table['flags']     <=    2) &
                                    (phot_table['type']      <=    2))]
        print('NIRCAM SHORT')

    if filt_n in ['F277W', 'F335M', 'F444W']:
        # NIRCAM Filter Long
        phot_table1 = phot_table[ (phot_table['sharpness']**2 <= 0.02) &
                                (phot_table['obj_crowd'] <=  0.5) &
                                (phot_table['flags']     <=    2) &
                                (phot_table['type']      <=    2)]

        phot_table2 = phot_table[~((phot_table['sharpness']**2 <= 0.02) &
                            (phot_table['obj_crowd'] <=           0.5) &
                            (phot_table['flags']     <=    2) &
                            (phot_table['type']      <=    2))]
        print('NIRCAM LONG')

    if filt_n in ['F435W', 'F555W', 'F814W']:
        # HST ACS
        phot_table1 = phot_table[ (phot_table['sharpness']**2 <   0.2) &
                                (phot_table['obj_crowd'] <       2.25) &
                                (phot_table['flags']     <=         2) &
                                (phot_table['type']      <=         2) &
                                (phot_table['SNR']       >          4)]

        phot_table2 = phot_table[ ~((phot_table['sharpness']**2 <  0.2) &
                            (phot_table['obj_crowd'] <            2.25) &
                            (phot_table['flags']     <=              2) &
                            (phot_table['type']      <=              2) &
                            (phot_table['SNR']       >               4))]
        print("HST ACS/WFC")

    if filt_n in ['F275W', 'F336W']:
        # HST WFC3
        phot_table1 = phot_table[(phot_table['sharpness']**2 < 0.15) &
                                (phot_table['obj_crowd'] <      1.3) &
                                (phot_table['flags']     <=       2) &
                                (phot_table['type']      <=       2) &
                                (phot_table['SNR']       >        5)]

        phot_table2 = phot_table[~((phot_table['sharpness']**2 < 0.15) &
                            (phot_table['obj_crowd'] <            1.3) &
                            (phot_table['flags']     <=             2) &
                            (phot_table['type']      <=             2) &
                            (phot_table['SNR']       >              5))]
        print("HST WFC3/UVIS")

    phot_table1['flag_phot'] = 1
    phot_table2['flag_phot'] = 0 

    phot_table = vstack([phot_table1, phot_table2])
    phot_table.write(f"../{out_dir}/{region}/{filt_n}/{filt_n}_photometry_filt.fits",overwrite=True)
    with open(f"../{out_dir}/{region}/{filt_n}/{filt_n}.reg",'w+') as f:
        f.write("""# Region file format: DS9 version 4.1
global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1
icrs\n""")
        temp = phot_table1[phot_table1['mag_err']<0.2]
        for row in temp:
            ra = np.round(row['ra'], 7)
            dec = np.round(row['dec'],7)
            f.write(f"""circle({ra},{dec},0.03")\n""")

In [ ]:
def sim_stars(out_dir, region, filt_n, d=10, mag=22, n=100):
    
    r = d/2
    
    if not os.path.exists(f"../PHOT_OUTPUT_m50/{region}/"):
        os.mkdir(f"../PHOT_OUTPUT_m50/{region}/")
        
    if not os.path.exists(f"../PHOT_OUTPUT_m50/{region}/{filt_n}"):
        os.mkdir(f"../PHOT_OUTPUT_m50/{region}/{filt_n}")
    else:
        os.system(f"rm -r ../PHOT_OUTPUT_m50/{region}/{filt_n}/*")
                
    hdu = fits.open(f"../{out_dir}/{region}/{filt_n}/data_conv.fits")[1]

    data_source = hdu.data
    
    if 'CD1_1' in hdu.header.keys() and 'CD1_2' in hdu.header.keys():
        pixel_scale = np.sqrt(hdu.header['CD1_1']**2 + hdu.header['CD1_2']**2)*3600
        
    elif 'CDELT1' in hdu.header.keys():
        pixel_scale = hdu.header['CDELT1']*3600
        
    phot_table = Table.read(f"../{out_dir}/{region}/{filt_n}/{filt_n}_photometry_filt.fits")

    #psf = np.median(fits.open(f'../data/PSF/epsf/{filt_n}/snap_test_psf.fits')[0].data, axis=(1,0))
    psf = fits.open(f'../data/PSF/epsf/{filt_n}/snap_test_psf.fits')[0].data[0,0]
    psf = psf.reshape(51,5,51,5).sum(axis=(1,3))
    psf /= psf.sum()

    hdu = fits.PrimaryHDU(psf)
    hdul = fits.HDUList([hdu])
    hdul.writeto('psf.fits', overwrite=True)

    tel_params ={
                'aperture'       : 650,
                'pixel_scale'    : pixel_scale,
                'psf_file'       : f'psf.fits',
                'response_funcs' :  [ f'{data_path}/INSIST/UV/Coating.dat,5,100',   # 6 mirrors
                                    ],
                 'coeffs'        : 0.5 ,
                 'theta'         : 0
                }

    df = phot_table[ (phot_table['SNR']<50) ][['ra', 'dec',f'mag_vega_{filt_n}', 'flux','flag_phot']].to_pandas()

    df = df[df['flag_phot']==1]
    zero_flux = (df['flux']/10**(-0.4*df[f'mag_vega_{filt_n}'])).mean()
    zp = 2.5*np.log10(zero_flux)
    
    dZP = {'F115W' : 0.5,
           'F150W' : 0.3,
           'F200W' : 0.1,
           
           'F435W' : 0.3,
           'F555W' : 0.3,
           'F814W' : 0.6,
           
           'F275W' : 0.9,
           'F336W' : 0.7
           
            }
    
    zero_flux = 10**(0.4*(zp + dZP[filt_n]))
    
    df = df.rename(columns = {f'mag_vega_{filt_n}': 'mag'})
    sim = pt.Imager(df=df, tel_params=tel_params, exp_time=2000,
                    n_x=data_source.shape[0], n_y=data_source.shape[1])
                          
    sim.shot_noise = False
                          
    det_params = {'shot_noise' :  'Poisson',
                  'qe_response': [],
                  'qe_mean'    : 1,
                  'G1'         :  1,
                  'bias'       : 10,
                  'PRNU_frac'  :  0.25/100,
                  'DCNU'       :  0.1/100,
                  'RN'         :  3,
                  'T'          :  218,
                  'DN'         :  0.01/100
                  }

    sim(det_params=det_params)
    
    r_pix = r/pixel_scale
    
    x_min  = sim.n_x_sim/2 - r_pix + 26
    x_max  = sim.n_x_sim/2 + r_pix - 26
    
    y_min  = sim.n_y_sim/2 - r_pix + 26
    y_max  = sim.n_y_sim/2 + r_pix - 26
    
    x = np.linspace(x_min,x_max, int(n**0.5))
    y = np.linspace(y_min,y_max, int(n**0.5))
    
    x, y = np.meshgrid(x, y)
    
    x = x.ravel()
    y = y.ravel()
    
    mag_ = np.ones(n)*mag
    
    df_add  = pd.DataFrame(zip(x,y,mag_), columns = ['x','y','mag'])
    out_img = sim.add_stars(data_source, zero_flux, df_add)

    hdu = fits.open(f"../{out_dir}/{region}/{filt_n}/data_conv.fits")
    
    hdu[1].data = out_img
    wcs = WCS(hdu[1].header)
    
    dx = {'F115W' : -1,
          'F150W' : 0,
          'F200W' : 0,
          'F435W' : 0,
          'F555W' : 0,
          'F814W' : 0,
          'F275W' : -1,
          'F336W' : -1}
    
    dy = {'F115W' : -1,
          'F150W' : 0,
          'F200W' : 0,
          'F435W' : -1,
          'F555W' : -1,
          'F814W' : -1,
          'F275W' : -1,
          'F336W' : -1}
            
    coords = np.array(wcs.array_index_to_world_values(y - 50 + dy[filt_n],
                                                      x - 50 + dx[filt_n]))
    
    df_add['x'] = x - 49 
    df_add['y'] = y - 49
    
    df_add['ra'] = coords[0,:]
    df_add['dec'] = coords[1,:]
    
    df_add = Table.from_pandas(df_add)
    
    df_add.write(f"../PHOT_OUTPUT_m50/{region}/{filt_n}/add_stars.fits", overwrite=True)
    hdu.writeto(f"../PHOT_OUTPUT_m50/{region}/{filt_n}/data_conv.fits", output_verify='ignore',
            overwrite=True)

In [ ]:
def comp_base(out_dir, region, filt_n, r=0.06, mag=22):
    in1 = f"../PHOT_OUTPUT_m50/{region}/{filt_n}/add_stars.fits"
    in2 = f"../PHOT_OUTPUT_m50/{region}/{filt_n}/{filt_n}_photometry_filt.fits"

    out_t = f"../PHOT_OUTPUT_m50/{region}/{filt_n}/matched_t.csv"
    out = f"../PHOT_OUTPUT_m50/{region}/{filt_n}/matched.csv"

    #os.system(f"topcat -stilts tskymatch2 in1={in1} in2={in2} out={out} \
     #             ra1=ra dec1=dec ra2=ra dec2=dec error={r}") join=1and2 find=all 
    
    os.system(f"""topcat -stilts tmatch2 in1={in1} in2={in2} \
               matcher=sky+1d params='{r} 0.5' \
               values1='ra dec mag' values2='ra dec mag_vega_{filt_n}'  \
               out={out_t}""")
       
    df_match = pd.read_csv(f"../PHOT_OUTPUT_m50/{region}/{filt_n}/matched_t.csv") 
    
    if len(df_match)>0:
        os.system(f"""topcat -stilts tpipe in={out_t} \
                        cmd='select "mag_err<=0.2"' \
                        out={out}""")

        df_match = pd.read_csv(f"../PHOT_OUTPUT_m50/{region}/{filt_n}/matched.csv") 
            
    with open(f"../PHOT_OUTPUT_m50/{region}/{filt_n}/{filt_n}_match_0.reg",'w+') as f:
        f.write("""# Region file format: DS9 version 4.1
global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1
icrs\n""")
        for i, row in df_match[df_match['flag_phot']==0].iterrows():
            ra = np.round(row['ra_1'], 7)
            dec = np.round(row['dec_1'],7)
            f.write(f"""circle({ra},{dec},0.06")\n""")
        
    df_match = df_match[df_match['flag_phot']==1]
    
    with open(f"../PHOT_OUTPUT_m50/{region}/{filt_n}/{filt_n}_match_1.reg",'w+') as f:
        f.write("""# Region file format: DS9 version 4.1
global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1
icrs\n""")
        for i, row in df_match.iterrows():
            ra = np.round(row['ra_1'], 7)
            dec = np.round(row['dec_1'],7)
            f.write(f"""circle({ra},{dec},0.06")\n""")
            
    df_match1 = df_match[abs(df_match['mag'] - df_match[f'mag_vega_{filt_n}'])<0.5]
    df_match2 = df_match[abs(df_match['mag'] - df_match[f'mag_vega_{filt_n}'])>=0.5]
    
    with open(f"../PHOT_OUTPUT_m50/{region}/{filt_n}/{filt_n}_match_2.reg",'w+') as f:
        f.write("""# Region file format: DS9 version 4.1
global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1
icrs\n""")
        for i, row in df_match1.iterrows():
            ra = np.round(row['ra_1'], 7)
            dec = np.round(row['dec_1'],7)
            f.write(f"""circle({ra},{dec},0.06")\n""")
            
    with open(f"../PHOT_OUTPUT_m50/{region}/{filt_n}/{filt_n}_match_3.reg",'w+') as f:
        f.write("""# Region file format: DS9 version 4.1
global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1
icrs\n""")
        for i, row in df_match2.iterrows():
            ra = np.round(row['ra_1'], 7)
            dec = np.round(row['dec_1'],7)
            f.write(f"""circle({ra},{dec},0.06")\n""")
    
    print(mag, len(df_match1))
    
    return len(df_match1)

In [ ]:
def comp_fit(out_dir, region, filt_n, x, y):
    
    init = pritchet()
    fit = fitting.LevMarLSQFitter()
    offset = y.max()
    model = fit(init, x, y/offset)

    fig, ax = plt.subplots(figsize=(9,7))
    ax.scatter(x,y)

    ax.plot(x, model(x)*offset, '--r')
    ax.set_xlabel('mags')
    ax.set_ylabel(r'$N_{out}/N_{in}$')
    ax.set_title(f"{region} | {filt_n} | "+ r"$\alpha =$" + f" {np.round(model.alpha.value,2)}" + r" | $m_{50}=$" + f"{np.round(model.m_50.value,2)}")

    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())

    ax.tick_params(which='both', width=2,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=7,direction="in")
    ax.tick_params(which='minor', length=4, color='black',direction="in")
    ax.set_ylim(0,1)
    fig.savefig(f"../{out_dir}/{region}/{filt_n}/{filt_n}_comp_{region}.png")
    plt.close(fig)

In [ ]:
@models.custom_model
def pritchet(m,alpha=0.5,m_50=30):
    return 0.5*(1 - alpha*(m - m_50)/np.sqrt(1 + alpha**2*(m-m_50)**2))

# **Run**

## **Bubbles**

In [ ]:
regions = { 'bubble': {'ra': 24.1859050, 'dec': 15.7726111},
            'bkg1':{'ra': 24.1918038, 'dec':15.7600032},
            'bkg2': {'ra': 24.1787197, 'dec': 15.7543227},
            'bkg3': {'ra': 24.1728133, 'dec': 15.7669357},
            #'epsf1': {'ra':24.2006271, 'dec': 15.7515889]
            'epsf2':{'ra': 24.2160615, 'dec': 15.7497349}
          }

## **Radial Completeness**

In [ ]:
regions = {'reg_0': {'ra': 24.1738983, 'dec': 15.783654299999997},
 'reg_1': {'ra': 24.178758715702823, 'dec': 15.773271121955174},
 'reg_2': {'ra': 24.183618633437526, 'dec': 15.762887836899594},
 'reg_3': {'ra': 24.188478053210716, 'dec': 15.752504445739396},
 'reg_4': {'ra': 24.193336975029013, 'dec': 15.742120949380759}}

## **Full FoV**

In [ ]:
regions = {'reg_0': {'ra': 24.170948143687465, 'dec': 15.831047906469903},
 'reg_1': {'ra': 24.160420209506793, 'dec': 15.826484636102231},
 'reg_2': {'ra': 24.14989275031864, 'dec': 15.821920858004587},
 'reg_3': {'ra': 24.139365766801628, 'dec': 15.81735657269709},
 'reg_4': {'ra': 24.175690821386876, 'dec': 15.820919016038717},
 'reg_5': {'ra': 24.16516330642193, 'dec': 15.81635597332094},
 'reg_6': {'ra': 24.154636266086623, 'dec': 15.811792423204704},
 'reg_7': {'ra': 24.144109701059556, 'dec': 15.807228366210039},
 'reg_8': {'ra': 24.18043302500756, 'dec': 15.810790020378326},
 'reg_9': {'ra': 24.16990592937138, 'dec': 15.806227205318896},
 'reg_10': {'ra': 24.15937930800206, 'dec': 15.801663883192525},
 'reg_11': {'ra': 24.148853161578113, 'dec': 15.797100054519241},
 'reg_12': {'ra': 24.185174754555632, 'dec': 15.80066092032992},
 'reg_13': {'ra': 24.174648078361198, 'dec': 15.796098332937296},
 'reg_14': {'ra': 24.164121876070894, 'dec': 15.79153523880932},
 'reg_15': {'ra': 24.15359614836323, 'dec': 15.7869716384659},
 'reg_16': {'ra': 24.18991601003724, 'dec': 15.790531716734755},
 'reg_17': {'ra': 24.179389753397466, 'dec': 15.785969357017382},
 'reg_18': {'ra': 24.168863970299206, 'dec': 15.78140649089625},
 'reg_19': {'ra': 24.158338661420906, 'dec': 15.776843118891206},
 'reg_20': {'ra': 24.194656791458534, 'dec': 15.780402410434041},
 'reg_21': {'ra': 24.1841309544863, 'dec': 15.775840278400347},
 'reg_22': {'ra': 24.173605590693008, 'dec': 15.771277640294525},
 'reg_23': {'ra': 24.1630807007571, 'dec': 15.766714496636352},
 'reg_24': {'ra': 24.19939709882576, 'dec': 15.770273002269},
 'reg_25': {'ra': 24.188871681633834, 'dec': 15.76571109792742},
 'reg_26': {'ra': 24.178346737258398, 'dec': 15.761148687845369},
 'reg_27': {'ra': 24.167822266377847, 'dec': 15.756585772542522},
 'reg_28': {'ra': 24.204136932145065, 'dec': 15.760143493080896},
 'reg_29': {'ra': 24.193611934846224, 'dec': 15.755581816439825},
 'reg_30': {'ra': 24.18308741000148, 'dec': 15.751019634389976},
 'reg_31': {'ra': 24.172563358289203, 'dec': 15.746456947450959},
 'reg_32': {'ra': 24.208876291422733, 'dec': 15.750013883710945},
 'reg_33': {'ra': 24.198351714129657, 'dec': 15.74545243477879},
 'reg_34': {'ra': 24.18782760892838, 'dec': 15.740890480769561},
 'reg_35': {'ra': 24.17730397649722, 'dec': 15.73632802220285}}

## **DOLPHOT Photometry**

In [ ]:
out_dir = 'PHOT_OUTPUT_r41'

if not os.path.exists(f"../{out_dir}/"):
    os.mkdir(f"../{out_dir}/")

In [ ]:
d = 41

for region in list(regions.keys())[1:]:
    print(region)
    
    for filt_n in ['F115W','F150W','F200W']:   
            NIRCAM_phot(out_dir, region, filt_n, d,
                    skip_phot=False, comp=True) 
    """
    for filt_n in ['F435W', 'F555W','F814W']:        
        acs_phot(out_dir, region, filt_n, d, 
                 skip_phot=False, comp=False)  
            
    if region != 'bkg1':
        for filt_n in ['F275W','F336W']:        
            wfc3_phot(out_dir, region, filt_n, d, 
                 skip_phot=False, comp=False)"""  

.....
197627 found
44879 stars eliminated
2312 stars eliminated
1122 stars eliminated
830 stars eliminated
561 stars eliminated
341 stars eliminated
215 stars eliminated
135 stars eliminated
74 stars eliminated
53 stars eliminated
28 stars eliminated
15 stars eliminated
7 stars eliminated
146 PSF stars; 15925 neighbors
Central pixel PSF adjustments:
image 1: 146 stars, -0.000900
77 stars eliminated
324 stars eliminated
265 stars eliminated
136 stars eliminated
18 stars eliminated
25 stars eliminated
17 stars eliminated
2 stars eliminated
Aperture corrections:
image 1: 200 total aperture stars
  200 stars used, -0.008 (-0.009 +/- 0.000, 0.001)
  139 stars used, -0.017 (-0.019 +/- 0.000, 0.002)
  200 stars used, -0.235 (-0.236 +/- 0.000, 0.001)


In [ ]:
for filt_n in ['F150W','F200W']:
    fs = glob.glob(f"../PHOT_OUTPUT_r40.5/*/{filt_n}/{filt_n}_photometry_filt.fits")
    fs = sorted(fs, key=lambda x: int(x.split('reg_')[-1].split('/')[0]))
    
    in1 = fs[0]
    for f in fs[1:]:
        in2 = f
        out1 = f"../PHOT_OUTPUT_r40.5/t1.fits"
        out2 = f"../PHOT_OUTPUT_r40.5/t2.fits"
        
        out3 = f"../PHOT_OUTPUT_r40.5/t3.fits"
        out4 = f"../PHOT_OUTPUT_r40.5/t4.fits"
        
        out5 = f"../PHOT_OUTPUT_r40.5/{filt_n}_photometry.fits"

        os.system(f"""topcat -stilts tpipe in={in1} \
            cmd='select "flag_phot==1 && mag_err<=0.2"' \
            out={out1}""")

        os.system(f"""topcat -stilts tpipe in={in2} \
        cmd='select "flag_phot==1 && mag_err<=0.2"' \
        out={out2}""")

        os.system(f"topcat -stilts tskymatch2 in1={out1} in2={out2} out={out3} \
                        join=1xor2 \
                     ra1=ra dec1=dec ra2=ra dec2=dec error=0.06")

        os.system(f"topcat -stilts tskymatch2 in1={out1} in2={out2} out={out4} \
                 ra1=ra dec1=dec ra2=ra dec2=dec error=0.06")

        tab = Table.read(out3)

        cols = [i.split('_1')[0] for i in tab.keys()[:27]]
        col1 = tab.keys()[:27]
        col2 = tab.keys()[27:]
        tab1 = tab[tab['ext_1']==1.0][col1]
        tab2 = tab[tab['ext_2']==1.0][col2]

        tab1.rename_columns(tab1.keys(), cols)
        tab2.rename_columns(tab2.keys(), cols)

        tab3 = vstack([tab1,tab2])
        tab3 = Table.read(out4)    
        x = tab3[f'mag_vega_{filt_n}_1']
        y = tab3[f'mag_vega_{filt_n}_2']

        plt.scatter(x,y)
        plt.show()

        rows = []
        for i in range(len(tab3)):
            row = tab3[i:i+1]
            if row['mag_err_1']<=row['mag_err_2']:
                row.rename_columns(col1,cols)
            else:
                row.rename_columns(col2,cols)
            rows.append(row[cols])

        tab4 = vstack(rows)
        tab5 = vstack([tab1,tab2,tab4])
        tab5.write(out5, overwrite=True) 
        in1 = out5

In [ ]:
for region in regions.keys():
    filters = ['F115W', 'F150W', 'F200W']
    for filt in itertools.combinations(filters,2):

        in1 = f"../{out_dir}/{region}/{filt[0]}/{filt[0]}_photometry_filt.fits"
        in2 = f"../{out_dir}/{region}/{filt[1]}/{filt[1]}_photometry_filt.fits"
        out1 = f"../{out_dir}/{region}/t1.fits"
        out2 = f"../{out_dir}/{region}/t2.fits"

        out = f"../{out_dir}/{region}/{filt[0].lower()}_{filt[1].lower()}.fits"

        os.system(f"""topcat -stilts tpipe in={in1} \
                    cmd='select "flag_phot==1 && mag_err<=0.2"' \
                    out={out1}""")

        os.system(f"""topcat -stilts tpipe in={in2} \
                    cmd='select "flag_phot==1 && mag_err<=0.2"' \
                    out={out2}""")

        os.system(f"topcat -stilts tskymatch2 in1={out1} in2={out2} out={out} \
                      ra1=ra dec1=dec ra2=ra dec2=dec error=0.06")
        
    filters = ['F435W', 'F555W', 'F814W']   
    for filt in itertools.combinations(filters,2):

        in1 = f"../{out_dir}/{region}/{filt[0]}/{filt[0]}_photometry_filt.fits"
        in2 = f"../{out_dir}/{region}/{filt[1]}/{filt[1]}_photometry_filt.fits"
        out1 = f"../{out_dir}/{region}/t1.fits"
        out2 = f"../{out_dir}/{region}/t2.fits"

        out = f"../{out_dir}/{region}/{filt[0].lower()}_{filt[1].lower()}.fits"

        os.system(f"""topcat -stilts tpipe in={in1} \
                    cmd='select "flag_phot==1 && mag_err<=0.2"' \
                    out={out1}""")

        os.system(f"""topcat -stilts tpipe in={in2} \
                    cmd='select "flag_phot==1 && mag_err<=0.2"' \
                    out={out2}""")

        os.system(f"topcat -stilts tskymatch2 in1={out1} in2={out2} out={out} \
                      ra1=ra dec1=dec ra2=ra dec2=dec error=0.1")
 
    filters = ['F275W', 'F336W']   
    if region!='bkg1':
        for filt in itertools.combinations(filters,2):

            in1 = f"../{out_dir}/{region}/{filt[0]}/{filt[0]}_photometry_filt.fits"
            in2 = f"../{out_dir}/{region}/{filt[1]}/{filt[1]}_photometry_filt.fits"
            out1 = f"../{out_dir}/{region}/t1.fits"
            out2 = f"../{out_dir}/{region}/t2.fits"

            out = f"../{out_dir}/{region}/{filt[0].lower()}_{filt[1].lower()}.fits"

            os.system(f"""topcat -stilts tpipe in={in1} \
                        cmd='select "flag_phot==1 && mag_err<=0.2"' \
                        out={out1}""")

            os.system(f"""topcat -stilts tpipe in={in2} \
                        cmd='select "flag_phot==1 && mag_err<=0.2"' \
                        out={out2}""")

            os.system(f"topcat -stilts tskymatch2 in1={out1} in2={out2} out={out} \
                          ra1=ra dec1=dec ra2=ra dec2=dec error=0.08")


In [ ]:
for region in regions:
    filters = ['F115W', 'F150W', 'F200W']
    
    in1 = f"../{out_dir}/{region}/{filters[0]}/{filters[0]}_photometry_filt.fits"
    in2 = f"../{out_dir}/{region}/{filters[1].lower()}_{filters[2].lower()}.fits"

    out = f"../{out_dir}/{region}/{filters[0].lower()}_{filters[1].lower()}_{filters[2].lower()}.fits"

    os.system(f"topcat -stilts tskymatch2 in1={in1} in2={in2} out={out} \
                  ra1=ra dec1=dec ra2=ra_1 dec2=dec_1 error=0.06")
      
    filters = ['F435W', 'F555W', 'F814W']  
    
    in1 = f"../{out_dir}/{region}/{filters[0]}/{filters[0]}_photometry_filt.fits"
    in2 = f"../{out_dir}/{region}/{filters[1].lower()}_{filters[2].lower()}.fits"

    out = f"../{out_dir}/{region}/{filters[0].lower()}_{filters[1].lower()}_{filters[2].lower()}.fits"

    os.system(f"topcat -stilts tskymatch2 in1={in1} in2={in2} out={out} \
                  ra1=ra dec1=dec ra2=ra_1 dec2=dec_1 error=0.1")

In [ ]:
for region in regions:
    filters = ['F115W', 'F150W', 'F200W']

    in1 = f"../{out_dir}/{region}/{filters[0].lower()}_{filters[1].lower()}_{filters[2].lower()}.fits"

    filters = ['F435W', 'F555W', 'F814W']  
 
    in2 = f"../{out_dir}/{region}/{filters[0].lower()}_{filters[1].lower()}_{filters[2].lower()}.fits"
    
    out = f"../{out_dir}/{region}/f115w_f150w_f200w_f435w_f555w_f814w.fits"
    
    os.system(f"topcat -stilts tskymatch2 in1={in1} in2={in2} out={out} \
                  ra1=ra_1 dec1=dec_1 ra2=ra_1 dec2=dec_1 error=0.1")
    
    if region != 'bkg1':
        in1 = f"../{out_dir}/{region}/f275w_f336w.fits"
        in2 = out

        out = f"../{out_dir}/{region}/f115w_f150w_f200w_f435w_f555w_f814w_f275w_f336w.fits"

        os.system(f"topcat -stilts tskymatch2 in1={in1} in2={in2} out={out} \
                      ra1=ra_1 dec1=dec_1 ra2=ra_1_1 dec2=dec_1_1 error=0.1")

# **Analysis**

In [ ]:
def gen_CMD(filt1='f115w', filt2='f150w',name=None, Av=0.19, r=333, r_in=None, r_out=None, dismod=29.46,
           x_cen=1594/2, y_cen=1594/2, xlims=[-0.5,2.5], ylims=[18,28],
           cmd=None, out_dir='PHOT_OUTPUT_EPSF', Av_ = 3, Av_x=2, Av_y=19,
           flag=111, gen_kde=False, add_ref=False, regions=['bubble','bkg1', 'bkg2','bkg3'],add_ext=''):
    
    fig = plt.figure(figsize=(len(regions)*7,7))
    for i,inp in enumerate(regions):
        
        ax = fig.add_subplot(1,len(regions),int(i+1))
        
        if name is None:
            tab_bub = Table.read(f"../{out_dir}/{inp}/{filt1}_{filt2}{add_ext}.fits")
        else:
            tab_bub = Table.read(f"../{out_dir}/{inp}/{name}.fits")
            
        tab_bub = tab_bub[(tab_bub['mag_err_1']<mag_err_lim) 
                          & (tab_bub['mag_err_2']<mag_err_lim) ]
        
        if 'flag_phot_1' in tab_bub.keys() and 'flag_phot_2' in tab_bub.keys():   
            if flag>0:
                tab_bub = tab_bub[tab_bub['flag_phot_1']== (flag-100)//10]
                tab_bub = tab_bub[tab_bub['flag_phot_2']== (flag-100)%10]

        AF1 =  Av_dict[filt1]*Av
        AF2 =  Av_dict[filt2]*Av

        tab_bub['r'] = np.sqrt( (tab_bub['x_1'] - x_cen )**2 +  (tab_bub['y_1'] - y_cen)**2 )
        
        if r_out is None or r_in is None :
            r_in = 0
            r_out = r
            
        tab_bub = tab_bub[ (tab_bub['r']>=r_in) & (tab_bub['r'] < r_out)]

        x = tab_bub[f'mag_vega_{filt1.upper()}'] - tab_bub[f'mag_vega_{filt2.upper()}']
        y = tab_bub[f'mag_vega_{filt2.upper()}'] 
        x = x.value.astype(float)
        y = y.value.astype(float)
        
        n_sources = len(x)
        
        if gen_kde:

            # Peform the kernel density estimate
            xx, yy = np.mgrid[xlims[0]:xlims[1]:100j, ylims[0]:ylims[1]:100j]
            positions = np.vstack([xx.ravel(), yy.ravel()])
            values = np.vstack([x, y])

            kernel = gaussian_kde(values, bw_method=0.075)
            f = np.reshape(kernel(positions), xx.shape)

            f = f.T
            img = ax.imshow(f, cmap='jet', 
                          extent=[xlims[0], xlims[1], 
                                  ylims[0], ylims[1]],
                           interpolation='nearest', aspect='auto')
        else:
            ax.scatter(x,y, s=0.1, color='black')

        ax.set_xlabel(f"{filt1.upper()} - {filt2.upper()}")
        ax.set_ylabel(filt2.upper())

        ref = tab_bub[f'mag_vega_{filt2.upper()}']
        ref_new = np.arange(np.ceil(y.min()),np.floor(y.max()),0.5)
        mag_err1 = interp1d(ref, tab_bub['mag_err_1'])(ref_new)
        mag_err2 = interp1d(ref, tab_bub['mag_err_2'])(ref_new)

        col_err = np.sqrt(mag_err1**2 + mag_err2**2)

        x = x.min() - 0.25 + 0*ref_new
        y = ref_new
        yerr = mag_err1
        xerr = col_err

        ax.errorbar(x, y,yerr,xerr,fmt='o', color = 'red', markersize=0.5, capsize=2)
        
        if cmd is not None:
            for age in ages_:

                t = cmd[np.round(cmd['logAge'],1)==age].copy()
                met = np.array(cmd['MH'])[0]
                x =  (t[f'{filt1.upper()}mag'] + AF1) - (t[f'{filt2.upper()}mag'] + AF2)
                y =  t[f'{filt2.upper()}mag'] + AF2 + dismod
                ax.plot(x,y,alpha=0.5)
        else: 
            met = None

        ax.set_ylim(ylims[0], ylims[1])
        ax.set_xlim(xlims[0], xlims[1])  
        ax.invert_yaxis()
        ax.set_title(inp.capitalize() + f" | No of sources: {n_sources} | M/H: {met}", fontsize=15)
        if gen_kde:
            if cmd is not None:
                ax.legend(age_lin)
        
        else:
            ax.legend(['data'] + age_lin)
            
        if add_ref:
            ax.plot([xlims[0], xlims[1]],[25,25], '--r', zorder=200)
            ax.plot([xlims[0], xlims[1]],[24.5,24.5], '--r', zorder=200)
            ax.plot([xlims[0], xlims[1]],[23,23], '--r', zorder=200)
            
        ax.xaxis.set_major_locator(AutoLocator())
        ax.xaxis.set_minor_locator(AutoMinorLocator())

        ax.yaxis.set_major_locator(AutoLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())

        ax.tick_params(which='both', length=7,direction="in", bottom=True, top=True,left=True, right=True)
        ax.tick_params(which='minor', length=4)

        AF1_ =  Av_dict[filt1]*Av_
        AF2_ =  Av_dict[filt2]*Av_

        dx = AF1_ - AF2_
        dy = AF1_

        ax.annotate('', xy=(Av_x, Av_y),
                     xycoords='data',
                     xytext=(Av_x+dx, Av_y+dy),
                     textcoords='data',
                     arrowprops=dict(arrowstyle= '<|-',
                                     color='black',
                                     lw=0.5,
                                     ls='-')
                   )
            
        ax.annotate(f'Av = {Av_}', xy=(Av_x-0.1, Av_y-0.1))

    return fig, ax

In [ ]:
def gen_CMD_cut( filt1='f115w', filt2='f150w',inp='bubble', name=None, Av=0.19, r=333,
                r_in=None, r_out=None,
                dismod=29.46, x_cen=1594/2, y_cen=1594/2, cmd=None,
                out_dir='PHOT_OUTPUT_EPSF', xlims=[-0.5,2.5], ylims=[18,30],
                age=9.0, cmd_xlo = None, cmd_xhi= None,
                l_lo = 22, l_hi=26.5, dl=0.5, Av_ = 3,
                Av_x=2, Av_y=22, flag=111, fit_isochrone=True):
    
    cmd_ylo = l_lo - dl
    cmd_yhi = l_hi + dl
    
    if name is None:
            tab_bub = Table.read(f"../{out_dir}/{inp}/{filt1}_{filt2}.fits")
    else:
        tab_bub = Table.read(f"../{out_dir}/{inp}/{name}.fits")
        
    tab_bub = tab_bub[(tab_bub['mag_err_1']<mag_err_lim) 
                    & (tab_bub['mag_err_2']<mag_err_lim) ]
    
    if 'flag_phot_1' in tab_bub.keys() and 'flag_phot_2' in tab_bub.keys():   
        if flag>0:
            tab_bub = tab_bub[tab_bub['flag_phot_1']== (flag-100)//10]
            tab_bub = tab_bub[tab_bub['flag_phot_2']== (flag-100)%10]
            

    AF1 =  Av_dict[filt1]*Av
    AF2 =  Av_dict[filt2]*Av
        
    tab_bub['r'] = np.sqrt( (tab_bub['x_1'] - x_cen )**2 +  (tab_bub['y_1'] - y_cen)**2 )

    if r_out is None or r_in is None :
        r_in = 0
        r_out = r

    tab_bub = tab_bub[ (tab_bub['r']>=r_in) & (tab_bub['r'] < r_out)]


    x = tab_bub[f'mag_vega_{filt1.upper()}'] - tab_bub[f'mag_vega_{filt2.upper()}']
    y = tab_bub[f'mag_vega_{filt2.upper()}'] 

    x = x.value.astype(float)
    y = y.value.astype(float)
    
    if cmd_xlo is None or cmd_xhi is None:
        cmd_xlo = x.mean() - 0.5

        cmd_xhi = x.mean() + 0.5

    fig = plt.figure(figsize=(7,7))
    ax = fig.add_subplot(1,1,1)

    theta = np.arctan(AF1/(AF1-AF2)) #
    
    ax.scatter(x,y, s=0.1, color='black')
    if cmd is not None:
        t = cmd[np.round(cmd['logAge'],1)==age].copy()
        met = np.array(cmd['MH'])[0]
        x_i =  (t[f'{filt1.upper()}mag'] + AF1) - (t[f'{filt2.upper()}mag'] + AF2)
        y_i =  t[f'{filt2.upper()}mag'] + AF2 + dismod
        
        x_i = np.array(x_i)
        y_i = np.array(y_i)
        
        x_iso = x_i.copy()
        y_iso = y_i.copy()
        
        x_i = x_i[np.where( (y_i>=cmd_ylo) & (y_i<=cmd_yhi))[0]]
        y_i = y_i[np.where( (y_i>=cmd_ylo) & (y_i<=cmd_yhi))[0]]

        y_i = y_i[np.where( (x_i>=cmd_xlo) & (x_i<=cmd_xhi))[0]]
        x_i = x_i[np.where( (x_i>=cmd_xlo) & (x_i<=cmd_xhi))[0]]
        
        ax.plot(x_i,y_i,alpha=0.8, zorder=200)
        

        ax.legend(['data', f'age = {age}'])

    x_l = np.linspace(0, 2)    

    # Bin mid points
    y_rgbn = np.arange(l_lo, l_hi, dl)
    
    y_rgb_mid = y_rgbn[:-1]+dl/2   
    
    if fit_isochrone:
        init = models.Linear1D()
        fit = fitting.LinearLSQFitter()
        model_iso = fit(init, y_i, x_i)
        
    else:
        model_iso = interp1d(y_i, x_i)
        
    x_rgb_mid = model_iso(y_rgb_mid)
    
    ax.scatter(x_rgb_mid, y_rgb_mid, c='r' ,zorder = 200)

    dats = []
    
    x0 = 1# x.mean()
    for y0 in y_rgbn[:-1]:

        # Extinction Vector
        x0 = model_iso(y0)
        y_Avl = y0 + np.tan(theta)*(x_l-x0)
        
        x01 = model_iso(y0 + dl)
        y_Avu = y0 + dl + np.tan(theta)*(x_l-x01)

        ax.plot(x_l,y_Avl, color='grey')
        ax.plot(x_l,y_Avu, color='grey')

        init = models.Linear1D()
        fit = fitting.LinearLSQFitter()
        
        model_Avl = fit(init, x_l, y_Avl)
        model_Avu = fit(init, x_l, y_Avu)

        c1 = (y>model_Avl(x)) & (y<=model_Avu(x))

        yn = y[np.where(c1)]
        xn = x[np.where(c1)]
        
        dat = np.array([xn, yn])
        dats.append(dat)
    
    ax.set_xlabel(f"{filt1.upper()} - {filt2.upper()}")
    ax.set_ylabel(filt2.upper())

    ax.set_ylim(ylims[0], ylims[1])
    ax.set_xlim(xlims[0], xlims[1])  
    ax.invert_yaxis()
    ax.set_title(inp.capitalize())

    ax.xaxis.set_major_locator(AutoLocator())
    ax.xaxis.set_minor_locator(AutoMinorLocator())

    ax.yaxis.set_major_locator(AutoLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())

    ax.tick_params(which='both', length=7,direction="in", bottom=True, top=True,left=True, right=True)
    ax.tick_params(which='minor', length=4)
    
    AF1_ =  Av_dict[filt1]*Av_
    AF2_ =  Av_dict[filt2]*Av_
    
    dx = AF1_ - AF2_
    dy = AF1_

    ax.annotate('', xy=(Av_x, Av_y),
                 xycoords='data',
                 xytext=(Av_x+dx, Av_y+dy),
                 textcoords='data',
                 arrowprops=dict(arrowstyle= '<|-',
                                 color='black',
                                 lw=0.5,
                                 ls='-')
               )

    ax.annotate(f'Av = {Av_}', xy=(Av_x-0.1, Av_y-0.1))
    return fig, ax, dats, x_rgb_mid, y_rgb_mid, y_rgbn

In [ ]:
Av_dict = { 
            'f275w': 2.02499,
            'f336w': 1.67536,
            'f435w': 1.33879,
            'f555w': 1.03065,
            'f814w': 0.59696,
            'f115w': 0.419,
            'f150w': 0.287,
            'f200w': 0.195,
          }

In [ ]:
out_dir = 'PHOT_OUTPUT_r25'